In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import duckdb
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str(datetime.now().date() - timedelta(days=1))
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")

Today's date: 2025-11-20


In [2]:
%run ./common_utils.ipynb

In [7]:
def create_actuals_tbl(con):
    df_lines = pd.read_csv(f"../tables/{year}/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[(df_lines.Date == now)]
    df_log = pd.read_csv(f"../tables/{year}/season_gamelogs.csv")
    df_log['Date'] = pd.to_datetime(df_log.Date)
    df_log = df_log[(df_log.Date == now)]
    df_log = df_log.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
    df_log['PR'] = df_log.PTS + df_log.REB 
    df_log['PA'] = df_log.PTS + df_log.AST
    df_log['RA'] = df_log.REB + df_log.AST
    df_log['PRA'] = df_log.PTS + df_log.REB + df_log.AST
    df = con.execute("""SELECT t2.*, t1.* EXCLUDE(Date, Team, Player, Pos) FROM df_log t1 JOIN df_lines t2 
                        ON t1.Team = t2.Team AND t1.Player = t2.Player""").fetchdf()
    cols = []
    for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
        df[f'{stat}_Result'] = np.where(df[stat] > df[f'{stat}_line'], 'O', 'U')
        df[f'{stat}_Result'] = np.where(df[f'{stat}_line'].isnull(), np.nan, df[f'{stat}_Result'])
        df[f'{stat}_Diff'] = df[stat] - df[f'{stat}_line']
        cols.append(stat)
        cols.append(f'{stat}_line')
        cols.append(f'{stat}_Diff')
        cols.append(f'{stat}_Result')
    df = df[['game_id', 'Date', 'Team', 'Player', 'Pos', 'MP'] + cols]
    
    df_stats = pd.read_csv(f"../tables/{year}/parlay_stats.csv")
    df_stats['Date'] = pd.to_datetime(df_stats.Date).astype(str)
    lines = []
    diffs = []
    for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
        lines.append(f'{stat}_line')
        diffs.append(f'{stat}_Diff')
    lines = str(lines).replace("[", "").replace("]", "")
    diffs = str(diffs).replace("[", "").replace("]", "")
    df_actuals = con.execute(f"""SELECT game_id, df.Date, df.Team, df.Player, df.Pos, MP, 
                            df_stats.* EXCLUDE(Date, Team, Player, {lines}, {diffs}), 
                            df.* EXCLUDE(game_id, Date, Team, Player, Pos, MP) FROM df JOIN df_stats 
                           ON df.Date = df_stats.Date AND df.Player = df_stats.Player
                           """).fetchdf()
    partition_save_df(df_actuals, f"../tables/{year}/parlay_actuals.csv")
    display(df_actuals)

    return df_actuals
                      
df = create_actuals_tbl(con)

../tables/2025/parlay_actuals.csv saved!


,game_id,Date,Team,Player,Pos,MP,B2B,Opp,Off_PTS,AVG_PTS_H2H,Def_PTS,Off_AST,AVG_AST_H2H,Def_AST,Off_REB,AVG_REB_H2H,Def_REB,Off_PR,AVG_PR_H2H,Def_PR,Off_PA,AVG_PA_H2H,Def_PA,Off_RA,AVG_RA_H2H,Def_RA,Off_PRA,AVG_PRA_H2H,Def_PRA,Off_TPM,AVG_TPM_H2H,Def_TPM,TPM_line,TPM_Diff,Off_STL,AVG_STL_H2H,Def_STL,STL_line,STL_Diff,Off_BLK,AVG_BLK_H2H,Def_BLK,BLK_line,BLK_Diff,Off_STL_BLK,AVG_STL_BLK_H2H,Def_STL_BLK,STL_BLK_line,STL_BLK_Diff,Off_L5_PTS,Def_L5_PTS,Off_L5_AST,Def_L5_AST,Off_L5_REB,Def_L5_REB,Off_L5_PR,Def_L5_PR,Off_L5_PA,Def_L5_PA,Off_L5_RA,Def_L5_RA,Off_L5_PRA,Def_L5_PRA,Off_L5_TPM,Def_L5_TPM,Off_L5_STL,Def_L5_STL,Off_L5_BLK,Def_L5_BLK,Off_L5_STL_BLK,Def_L5_STL_BLK,PTS,PTS_line,PTS_Diff,PTS_Result,AST,AST_line,AST_Diff,AST_Result,REB,REB_line,REB_Diff,REB_Result,PR,PR_line,PR_Diff,PR_Result,PA,PA_line,PA_Diff,PA_Result,RA,RA_line,RA_Diff,RA_Result,PRA,PRA_line,PRA_Diff,PRA_Result


In [5]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Player, Pos, MP, Opp, 
                            Off_{stat}, AVG_{stat}_H2H, Def_{stat}, {stat}_line, {stat}, {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL
                            ORDER BY {stat}_Diff DESC
                            LIMIT 15""").fetchdf())

,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PTS,AVG_PTS_H2H,Def_PTS,PTS_line,PTS,PTS_Diff,PTS_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_AST,AVG_AST_H2H,Def_AST,AST_line,AST,AST_Diff,AST_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_REB,AVG_REB_H2H,Def_REB,REB_line,REB,REB_Diff,REB_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PR,AVG_PR_H2H,Def_PR,PR_line,PR,PR_Diff,PR_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PA,AVG_PA_H2H,Def_PA,PA_line,PA,PA_Diff,PA_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_RA,AVG_RA_H2H,Def_RA,RA_line,RA,RA_Diff,RA_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PRA,AVG_PRA_H2H,Def_PRA,PRA_line,PRA,PRA_Diff,PRA_Result


In [6]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Player, Pos, MP, Opp, 
                            Off_{stat}, AVG_{stat}_H2H, Def_{stat}, {stat}_line, {stat}, {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL AND MP > 0
                            ORDER BY {stat}_Diff
                            LIMIT 15""").fetchdf())

,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PTS,AVG_PTS_H2H,Def_PTS,PTS_line,PTS,PTS_Diff,PTS_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_AST,AVG_AST_H2H,Def_AST,AST_line,AST,AST_Diff,AST_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_REB,AVG_REB_H2H,Def_REB,REB_line,REB,REB_Diff,REB_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PR,AVG_PR_H2H,Def_PR,PR_line,PR,PR_Diff,PR_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PA,AVG_PA_H2H,Def_PA,PA_line,PA,PA_Diff,PA_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_RA,AVG_RA_H2H,Def_RA,RA_line,RA,RA_Diff,RA_Result


,game_id,Date,Team,B2B,Player,Pos,MP,Opp,Off_PRA,AVG_PRA_H2H,Def_PRA,PRA_line,PRA,PRA_Diff,PRA_Result
